In [8]:
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
import gym
import math

env= gym.make('CartPole-v1')
n_bins  = (6, 12)
lower_bounds = [env.observation_space.low[2], -math.radians(50)]
upper_bounds = [env.observation_space.high[2], math.radians(50)]
est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
est.fit([lower_bounds, upper_bounds])
Q_table = np.zeros(n_bins + (env.action_space.n,))
done = False
lr = 0.1
df = 0.9
no_steps = 0
s = env.reset()
s = s[0]
fbin = est.transform([[s[2], s[3]]])[0]
current_state = (int(fbin[0]), int(fbin[1]))
while done == False:
    action = np.argmax(Q_table[current_state])
    s, reward, done, info, _ = env.step(action)
    fbin = est.transform([[s[2],s[3]]])[0]
    new_state = (int(fbin[0]), int(fbin[1]))
    next_action = np.argmax(Q_table[new_state])
    Q_table[current_state][action] = Q_table[current_state][action] + lr * \
    (reward + df * Q_table[new_state][next_action] - Q_table[current_state][action])
    current_state = new_state
    no_steps = no_steps + 1
print(f"no_steps = {no_steps}")
env.close()

no_steps = 9
